<a href="https://colab.research.google.com/github/aperfilyeva/anna_data690/blob/main/individual_project/Individualism_and_Suicides_class_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Individualism and Suicides

**Anna Perfilyeva**

Suicide is an important global health issue. It is estimated that 800,000 deaths per year occur due to suicide (1) and it is the fourth leading cause of death for youth 15-19 years old (2). Risk factors for suicide include mental health issues, alcohol and drug abuse, stress, abuse and trauma. Although there is a significant mental health aspect to this issue, social factors play a critical role in individuals’ risk of suicide. World Bank data offers a glimpse into how cultural factors are affecting this issue. In particular, I would like to look into how a country’s level of individualism affects suicide rates, using the World Bank data available at World Development Explorer (worlddev.xyz).

##Global Outlook

First let's take a look at countries with the highest and lowest suicide rates, expressed as the number of suicide deaths per 100,000 individuals in a given population, based on 2019 World Bank data:

In [20]:
import plotly.express as px
import plotly
import pandas as pd

data_url = 'https://raw.githubusercontent.com/aperfilyeva/anna_data690/main/individual_project/wdi_data.csv'

df = pd.read_csv(data_url)
#df.head(15)

In [21]:
#Top 10 - countries with highest suicide mortality- 2019
df_2019_sui = df.query("Year == 2019").query("indicator == 'SH.STA.SUIC.P5'").sort_values(by="value", ascending=False).head(10)
#df_2019_sui.head( )
fig = px.bar(data_frame=df_2019_sui, 
             x="Country Name", y="value",
             labels={"value":"2019 Suicide Rate (per 100,000 Population)","Country Name":""},
             color="Country Name",
             title="TOP 10 Countries by 2019 Suicide Mortality Rate",
             height=500,
             width=800,
             template=list(plotly.io.templates.keys())[5]
)
fig.update_layout(showlegend=False)

fig.show()

In [22]:
#Bottom 10-  countries with LOWEST suicide mortality- 2019
df_2019_sui = df.query("Year == 2019").query("indicator == 'SH.STA.SUIC.P5'").sort_values(by="value", ascending=True).head(10)

df_2019_sui_bottom=df_2019_sui.sort_values(by="value", ascending=False)

fig = px.bar(data_frame=df_2019_sui_bottom, 
             x="Country Name", y="value",
             labels={"value":"2019 Suicide Rate (per 100,000 Population)","Country Name":""},
             color="Country Name",
             title="BOTTOM 10 Countries by 2019 Suicide Mortality Rate",
             height=500,
             width=800,
             template=list(plotly.io.templates.keys())[5]
)
fig.update_layout(showlegend=False)

fig.show()

Across all nations, suicide rates range from less than 0.5 suicide deaths to over 70 deaths per 100,000 people. Lesotho, a country in Sub-Saharan Africa, has the largest suicide rate in the world, 72 persons per 100,000 commit suicide - more than double the incidence compared to the average rates in the other 9 top suicide countries. The primary reasons for this are thought to be extreme poverty and a high HIV prevalence. Notably, countries with lowest suicide rates are also predominantly lower-income economies.

## Suicide trends over time

To understand how suicide rates have changed over time, we will group countries into 5 income categories and start back at 2000. 


In [28]:
df_sui = df.query("indicator == 'SH.STA.SUIC.P5'").sort_values(by="value", ascending=True)
df_group_mean = df_sui.groupby(['Income Group', 'Year']).mean().reset_index().sort_values(by="Year", ascending=True)

#df_group_mean

fig = px.line(
    df_group_mean, 
    x="Year", 
    y="value",
    labels={"value":"Suicide Rate (per 100,000 Population)"},
    color="Income Group",
    height=500,
    width=800,
    #opacity=0.50,
    template=list(plotly.io.templates.keys())[5],
    title="Suicide Rates by Income Group from 2000 to 2019"
)

fig.update_layout(showlegend=True)
fig.update_xaxes(dtick=2)  # sets a tick mark at defined interval

fig.show()



##Correlation of Suicide Rates with Income and Health

Before we look into how individualism affects suicide rates, it is worth understanding how suicides correlate with country's income, represented in the data by GDP per capita, and the general health status, represented by life expectancy rates. We know that income is a powerful determinant of many social and health outcomes; higher incomes are typically associated with better outcomes. We would therefore expect that suicides are generally higher in lower income countries. Fitting a simple linear regression model on GDP and suicides data does no quite support this hypothesis: in fact, there is a slight upward trend in suicides as the country income increases. The P-value on the slope coefficient is relatively high, at 0.118, indicating that the positive correlation between suicide rates and a country's income is not statistically significant.

###Suicide Correlation with GDP

In [25]:
# Pivot the table to create a column for each indicator
df_pivot = df.pivot_table(index=["Year", "Country Code", "Country Name", "Region", "Income Group", "Lending Type"], 
                                    columns=["indicator"] , values="value")
df_pivot_2019=df_pivot.query("Year == 2019")

#OLS on Suicides vs GDP
fig = px.scatter(df_pivot_2019, x="NY.GDP.PCAP.PP.CD", 
                 y="SH.STA.SUIC.P5", 
                 trendline="ols", 
                 labels={"NY.GDP.PCAP.PP.CD":"2019 GDP per Capita (PPP)","SH.STA.SUIC.P5":"2019 Suicide Rate (per 100,000 Population)"},
                 height=500,
                 width=800,
                 template=list(plotly.io.templates.keys())[5],
                 title="Suicide Correlation with GDP")
fig.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [26]:
df_pivot.reset_index(inplace=True)
df_pivot.head()

indicator,Year,Country Code,Country Name,Region,Income Group,Lending Type,IDV,NY.GDP.PCAP.PP.CD,SH.STA.SUIC.P5,SP.DYN.LE00.IN,SP.POP.TOTL
0,2000,ABW,Aruba,Latin America & Caribbean,High income,Not classified,NaN,32026.972740,NaN,73.787,90866.0
1,2000,AFG,Afghanistan,South Asia,Low income,IDA,NaN,NaN,4.9,55.841,20779957.0
2,2000,AGO,Angola,Sub-Saharan Africa,Lower middle income,IBRD,NaN,3289.409549,8.7,46.522,16395477.0
3,2000,ALB,Albania,Europe & Central Asia,Upper middle income,IBRD,NaN,3860.804627,4.9,73.955,3089027.0
4,2000,AND,Andorra,Europe & Central Asia,High income,Not classified,NaN,NaN,NaN,NaN,65390.0


In [ ]:
df_pivot.reset_index(inplace=True)
df_pivot.head()


##Suicide Correlation with Life Expectancy

Similarly, looking at suicide rates across the life expectancy ranges, we would expect that more suicides tend to occur in countries where healthcare system is lacking, which affects both mental health and the general health status of individuals. In countries with successul healthcare systems and therefore higher life expectancy, we would expect suicide rates to be lower. Again, the simple linear regression of suicide rates against a country’s life expectancy does not quite confirm this. There in fact appears to be a small decrease in suicide rates as the country’s life expectancy increases; however, the P-value, at 0.082, shows a low statistical significance of this relationship.

In [33]:
#OLS on Suicides vs Life Expectancy
fig = px.scatter(df_pivot_2019, x="SP.DYN.LE00.IN", 
                 y="SH.STA.SUIC.P5", 
                 trendline="ols", 
                 labels={"SP.DYN.LE00.IN":"2019 Life expectancy at birth, total (years)","SH.STA.SUIC.P5":"2019 Suicide Rate (per 100,000 Population)"},
                 height=500,
                 width=800,
                 template=list(plotly.io.templates.keys())[5],
                 title="Suicide Correlation with Life Expectancy")

fig.show()

## How Individualistic Culture Affects Suicide

To better understand factors that lead to suicide, we will now look at society’s level of individualism vs. collectivism. World Bank defines individualism as a belief that government should not affect people’s choices. In day-to-day life we associate it with the high opportunity for self-realization and individual freedoms, and a lower reliance on the community. Individualistic cultures place more value on each person’s achievements, characteristics and concerns, whereas collectivist cultures place higher value on groups and communities, and tend to focus more on a person’s role in relation to other people, e.g. whether a person is a good friend, team player, coworker, etc. For reference, United States, Australia and United Kingdom rank highest on the individualism scale; Venezuela, Colombia and Indonesia rank lowest.

There are two different, in fact opposite, ways in which a society's level of individulaism can affect suicides: one hypothesis would be that higher individual freedoms make people happier, more self-fulfilled, and therefore reduce suicides. Another hypothesis is that a lower level of social connection in countries with higher individualis makes people feel more isolated and disconnected, leading to the increase in suicides. Plotting a simple linear regression of suicide rates against society's level of individualism, we see that there is a positive correlation between these variables, i.e. higher individualism is associated with higher suicide rates, supporting the second hypothese. Moreover, this relationship is statistically significant, with a P value of 0.012.  


In [ ]:
#OLS on Suicides vs Individualism
fig = px.scatter(df_pivot_2019, x="IDV", 
                 y="SH.STA.SUIC.P5", 
                 trendline="ols", 
                 labels={"IDV":"Individualism vs Collectivism (IDV)","SH.STA.SUIC.P5":"2019 Suicide Rate (per 100,000 Population)"},
                 height=500,
                 width=800,
                 template=list(plotly.io.templates.keys())[5],
                 title="Suicide Correlation with Individualism")
fig.show()

So, it appears that despite the many benefits that individualistic cultures offer, they are also associated with higher rates of suicides. It seems plausible that the lower level of community connection and social support results in a feeling of loneliness that can aggravate to where a person wants to end their life. Moreover, if we consider that an act of suicide is only the most visible and acute outcome of the mental pain and suffering an individual goes through, it’s likely that individualism is a factor in other mental health issues, such as stress and depression. It is therefore very important to create and strengthen communities and foster social support through schools, work organizations and places of worship, while respecting individual freedoms. 

 Sources:
 1. https://psycnet.apa.org/record/2018-49833-033
 2. https://www.who.int/news-room/fact-sheets/detail/suicide
